In [7]:
import pandas as pd

from datasetsforecast.long_horizon import LongHorizon

import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'



# Change this to your own data to try the model
Y_df, _, _ = LongHorizon.load(directory='./', group='ETTm2')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

n_time = len(Y_df.ds.unique())
val_size = int(.2 * n_time)
test_size = int(.2 * n_time)

Y_df.groupby('unique_id').head(2)

,unique_id,ds,y
0,HUFL,2016-07-01 00:00:00,-0.041413
1,HUFL,2016-07-01 00:15:00,-0.185467
57600,HULL,2016-07-01 00:00:00,0.040104
57601,HULL,2016-07-01 00:15:00,-0.214450
115200,LUFL,2016-07-01 00:00:00,0.695804
115201,LUFL,2016-07-01 00:15:00,0.434685
172800,LULL,2016-07-01 00:00:00,0.434430
172801,LULL,2016-07-01 00:15:00,0.428168
230400,MUFL,2016-07-01 00:00:00,-0.599211
230401,MUFL,2016-07-01 00:15:00,-0.658068


In [8]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

from neuralforecast.core import NeuralForecast
from neuralforecast.models import Autoformer
horizon = 96 # 24hrs = 4 * 15 min.
models = [#Informer(h=horizon,                 # Forecasting horizon
                # input_size=horizon,           # Input size
                # max_steps=1000,               # Number of training iterations
                # val_check_steps=100,          # Compute validation loss every 100 steps
                # early_stop_patience_steps=3), # Stop training if validation loss does not improve
          Autoformer(h=horizon,
                input_size=horizon,
                max_steps=1000,
                val_check_steps=100,
                early_stop_patience_steps=3),
          # PatchTST(h=horizon,
          #       input_size=horizon,
          #       max_steps=1000,
          #       val_check_steps=100,
          #       early_stop_patience_steps=3),
         ]

nf = NeuralForecast(
    models=models,
    freq='15min')

INFO:lightning_fabric.utilities.seed:Seed set to 1


In [9]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

Y_hat_df = nf.cross_validation(df=Y_df,
                               val_size=val_size,
                               test_size=test_size,
                               n_windows=None)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (mps), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type          | Params
------------------------------------------------
0 | loss          | MAE           | 0     
1 | padder_train  | ConstantPad1d | 0     
2 | scaler        | TemporalNorm  | 0     
3 | decomp        | SeriesDecomp  | 0     
4 | enc_embedding | DataEmbedding | 384   
5 | dec_embedding | DataEmbedding | 384   
6 | encoder       | Encoder       | 148 K 
7 | decoder       | Decoder       | 141 K 
------------------------------------------------
290 K     Trainable params
0         Non-trainable params
290 K     Total params
1.162     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

NotImplementedError: The operator 'aten::_fft_r2c' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

In [ ]:
from neuralforecast.losses.numpy import mae

mae_autoformer = mae(Y_hat_df['y'], Y_hat_df['Autoformer'])
print(f'Autoformer: {mae_autoformer:.3f}')